In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import nltk
import os
import lightgbm as lgb
import psutil

import matplotlib.pyplot as plt 
from functools import partial # to reduce df memory consumption by applying to_numeric

import warnings
warnings.filterwarnings('ignore')

In [2]:
cwd = os.getcwd()
cwd

'/home/ubuntu'

In [3]:
os.chdir('/home/ubuntu/data')

In [4]:
aisles = pd.read_csv('/home/ubuntu/data/aisles.csv', engine='c')
print('Total aisles: {}'.format(aisles.shape[0]))
aisles.head()

Total aisles: 134


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [5]:
departments = pd.read_csv('/home/ubuntu/data/departments.csv', engine='c')
print('Total departments: {}'.format(departments.shape[0]))

Total departments: 21


In [6]:
products = pd.read_csv('/home/ubuntu/data/products.csv', engine='c')
print('Total products: {}'.format(products.shape[0]))
products.head(5)

Total products: 49688


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [7]:
orders = pd.read_csv('/home/ubuntu/data/orders.csv', engine='c')
print('Total ordered products(prior): {}'.format(orders.shape[0]))
orders.eval_set.unique()
# orders.head()

Total ordered products(prior): 3421083


array(['prior', 'train', 'test'], dtype=object)

In [8]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [9]:
op_prior = pd.read_csv('/home/ubuntu/data/order_products__prior.csv', engine='c', 
                       dtype={'order_id': np.int32, 'product_id': np.int32, 
                              'add_to_cart_order': np.int16, 'reordered': np.int8})    
print('Total ordered products(train): {}'.format(op_prior.shape[0]))
op_prior.head()

Total ordered products(train): 32434489


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [10]:
op_train = pd.read_csv('/home/ubuntu/data/order_products__train.csv', engine='c', 
                       dtype={'order_id': np.int32, 'product_id': np.int32, 
                              'add_to_cart_order': np.int16, 'reordered': np.int8})    
print('Total ordered products(train): {}'.format(op_train.shape[0]))
op_train.head()

Total ordered products(train): 1384617


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [11]:
psutil.virtual_memory()

svmem(total=33737302016, available=32402788352, percent=4.0, used=940802048, free=31149486080, active=2057531392, inactive=286646272, buffers=153141248, cached=1493872640, shared=9175040)

In [12]:
# Adding user_id to op_train for merging purposes later
op_train = pd.merge(orders, op_train, how='inner', on='order_id')

In [13]:
psutil.virtual_memory()

svmem(total=33737302016, available=32200957952, percent=4.6, used=1142640640, free=30947610624, active=2260328448, inactive=286646272, buffers=153165824, cached=1493884928, shared=9175040)

In [14]:
op_prior = pd.merge(orders,op_prior, how="inner", on="order_id")

In [ ]:
op_prior.eval_set.unique()

In [ ]:
op_prior.columns 

In [ ]:
op_train.columns

In [ ]:
df_user = pd.DataFrame()
df_user['ave_days_bet_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
df_user['total_orders'] = orders.groupby('user_id', as_index=True).size().astype(np.int16)
df_user.head()

In [ ]:
df_user['user_id'] = df_user.index
op_prior_with_user_data = pd.merge(op_prior, df_user, how='inner', on='user_id')
# op_train_with_user_data = pd.merge(op_train, df_user, how='inner', on='user_id')
# data = op

In [ ]:
data['days_since_prior_order']=data['days_since_prior_order'].fillna(0)

In [ ]:
data.head()

In [ ]:
data.dropna()

In [ ]:
psutil.virtual_memory()

In [ ]:
op_train['eval_set'] = 'train'

In [ ]:
data = pd.merge(data, op_train['user_id'], how='left', on=['product_id','user_id'])

In [ ]:
df_train = data[data.eval_set == 'train']
df_test = data[data.eval_set == 'test']

In [ ]:
len(df_train)

In [ ]:
#data.eval_set.unique()
#df_train = df_train.drop(['order_id_y','add_to_cart_order_y','reordered_y'], axis=1)

target_train = df_train.loc['reordered_x']
#df_train = df_train[:, df_train.columns != 'reordered_x']

In [ ]:
df_train.head()

In [ ]:
data.shape
data.head()

In [ ]:
d_train = lgb.Dataset(df_train,
                      labels=target_train)

params = {
    'task': 'train',
    'boosting_type:' 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 96,
    'max_depth': 10,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5
    
}

ROUNDS = 100

mod = lgb.train(params, d_train, ROUNDS)